In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 45, 6, Finished, Available)

# Set up data configuration

In [31]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 13, Finished, Available)

# Reading in single parquet file

In [32]:
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 14, Finished, Available)

# Read the Reddit data

In [33]:
topic = ["FIFA"]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 15, Finished, Available)

In [34]:
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 16, Finished, Available)

In [35]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(topic))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 17, Finished, Available)

In [36]:
df_post = submissions_filtered.select("subreddit","title","selftext", "created_utc", "year", "month").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 18, Finished, Available)

In [37]:
df_post.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 19, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+
|subreddit|               title|            selftext|        created_utc|year|month|
+---------+--------------------+--------------------+-------------------+----+-----+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|
|     FIFA|should i select o...|           [removed]|2023-02-22 21:09:12|2023|    2|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|
|     FIFA|What’s a top/meta...|           [removed]|2023-02-22 21:17:18|2023|    2|
|     FIFA|Is there any reas...|For the past 10 d...|2023-02-22 21:24:30|2023|    2|
+---------+--------------------+--------------------+-------------------+----+-----+
only showing top 5 rows



In [38]:
from pyspark.sql.types import StructType, StructField, DateType
from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.functions import col, to_date, when, broadcast, udf, explode, length

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 20, Finished, Available)

In [39]:
df_post_date = df_post.withColumn("post_date", to_date(col("created_utc"), "yyyy-MM-dd"))
df_post_date.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 21, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|2023-02-22|
|     FIFA|should i select o...|           [removed]|2023-02-22 21:09:12|2023|    2|2023-02-22|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|2023-02-22|
|     FIFA|What’s a top/meta...|           [removed]|2023-02-22 21:17:18|2023|    2|2023-02-22|
|     FIFA|Is there any reas...|For the past 10 d...|2023-02-22 21:24:30|2023|    2|2023-02-22|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
only showing top 5 rows



In [18]:
df_post_date.printSchema()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 45, 23, Finished, Available)

root
 |-- subreddit: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- post_date: date (nullable = true)



In [40]:
filtered_df = df_post_date.filter(col("year") >= "2021")

# Show the result
filtered_df.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 22, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|2023-02-22|
|     FIFA|should i select o...|           [removed]|2023-02-22 21:09:12|2023|    2|2023-02-22|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|2023-02-22|
|     FIFA|What’s a top/meta...|           [removed]|2023-02-22 21:17:18|2023|    2|2023-02-22|
|     FIFA|Is there any reas...|For the past 10 d...|2023-02-22 21:24:30|2023|    2|2023-02-22|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
only showing top 5 rows



# spark NLP pipeline build

In [23]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, -1, Finished, Available)

Unrecognized options: 

In [24]:
!pip install spark-nlp

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 6, Finished, Available)

     |████████████████████████████████| 540 kB 10.2 MB/s eta 0:00:01


In [25]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 7, Finished, Available)

In [26]:
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 8, Finished, Available)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


# run the pipeline

In [41]:
filtered_df = filtered_df.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 23, Finished, Available)

In [42]:
senti_model = nlpPipeline.fit(filtered_df)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 24, Finished, Available)

In [43]:
senti_pred = senti_model.transform(filtered_df)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 25, Finished, Available)

In [44]:
senti_pred.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 26, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|            document| sentence_embeddings|           sentiment|
+---------+--------------------+--------------------+-------------------+----+-----+----------+--------------------+--------------------+--------------------+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|2023-02-22|[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA|should i select o...|           [removed]|2023-02-22 21:09:12|2023|    2|2023-02-22|[{document, 0, 8,...|                  []|                  []|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|2023-02-22|[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA|What’s a top/meta...|           [re

In [45]:
df_senti = senti_pred.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 27, Finished, Available)

In [46]:
df_senti = df_senti.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 28, Finished, Available)

In [47]:
df_senti.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 29, Finished, Available)

+---------+---------+
|subreddit|sentiment|
+---------+---------+
|     FIFA|      pos|
|     FIFA|      pos|
|     FIFA|      neg|
|     FIFA|      pos|
|     FIFA|      pos|
+---------+---------+
only showing top 5 rows



In [49]:
from pyspark.sql.functions import col, size

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 31, Finished, Available)

In [50]:
df_use = senti_pred.filter(size(col("sentence_embeddings")) != 0)
df_use.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 32, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|            document| sentence_embeddings|           sentiment|
+---------+--------------------+--------------------+-------------------+----+-----+----------+--------------------+--------------------+--------------------+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|2023-02-22|[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|2023-02-22|[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA|Is there any reas...|For the past 10 d...|2023-02-22 21:24:30|2023|    2|2023-02-22|[{document, 0, 63...|[{sentence_embedd...|[{category, 0, 63...|
|     FIFA|  flair passes trait|anyone know wh

In [51]:
df_filtered_use_final = df_use.drop(*("document", "sentence_embeddings", "sentiment"))
df_senti_use=df_senti.drop("subreddit")
df_filtered_use_final.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 33, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
|     FIFA|if you every get ...|                    |2023-02-22 21:07:53|2023|    2|2023-02-22|
|     FIFA| Team of the week 17|                    |2023-02-22 21:10:48|2023|    2|2023-02-22|
|     FIFA|Is there any reas...|For the past 10 d...|2023-02-22 21:24:30|2023|    2|2023-02-22|
|     FIFA|  flair passes trait|anyone know which...|2023-02-21 05:44:49|2023|    2|2023-02-21|
|     FIFA|Can’t believe wha...|                    |2023-02-21 05:13:52|2023|    2|2023-02-21|
+---------+--------------------+--------------------+-------------------+----+-----+----------+
only showing top 5 rows



In [54]:
from pyspark.sql.functions import monotonically_increasing_id

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 36, Finished, Available)

In [55]:
df_senti_text_use = df_filtered_use_final.withColumn("index", monotonically_increasing_id()).join(df_senti_use.withColumn("index", monotonically_increasing_id()),"index").drop("index")
df_senti_text_use.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 37, Finished, Available)

+---------+--------------------+--------------------+-------------------+----+-----+----------+---------+
|subreddit|               title|            selftext|        created_utc|year|month| post_date|sentiment|
+---------+--------------------+--------------------+-------------------+----+-----+----------+---------+
|     FIFA|Does anyone know ...|           [removed]|2022-11-28 08:06:17|2022|   11|2022-11-28|      pos|
|     FIFA|/r/FIFA: The Week...|It's not that we ...|2023-03-29 11:00:15|2023|    3|2023-03-29|      pos|
|     FIFA|It's not the firs...|           [removed]|2023-02-28 10:24:53|2023|    2|2023-02-28|      pos|
|     FIFA|Selling (fifa 23 ...|           [removed]|2023-03-29 11:41:15|2023|    3|2023-03-29|      pos|
|     FIFA|Is this a bug or ...|           [removed]|2022-11-28 08:01:10|2022|   11|2022-11-28|      pos|
+---------+--------------------+--------------------+-------------------+----+-----+----------+---------+
only showing top 5 rows



In [58]:
df_senti_text_save=df_senti_text_use.toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 46, 40, Submitted, Running)

In [ ]:
import os
CSV_DIR = os.path.join("Users/zx111/fall-2023-reddit-project-team-11/data", "csv")
df_senti_text_save.to_csv(f"{CSV_DIR}/FIFA_sentiment_text.csv", index=False)

StatementMeta(, , , Waiting, )